In [1]:
!pip install langchain-nvidia-ai-endpoints==0.3.5
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninstalling pillow-11.2.1:
      Successfully uninstalled pillow-11.2.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
import os, re, sqlite3, requests, json, numpy as np
from bs4 import BeautifulSoup
from dateutil import parser
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, NVIDIARerank
from langchain.schema import Document

In [ ]:
os.environ["NVIDIA_API_KEY"] = "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"

# List of incidents and corresponding news article

In [ ]:
incidents = {
    "Oregon_Rowena_Fire_Jun2025": [
       'https://apnews.com/article/oregon-wildfire-interstate-84-8d28833059cd46878080fd7e71444548',
       'https://www.opb.org/article/2025/05/14/an-architect-of-oregons-wildfire-map-on-why-he-now-supports-repealing-it/',
       'https://www.kgw.com/article/news/local/wildfire/oregon-state-fire-marshal-clears-misinformation/283-49abc8db-7d30-4bd9-8b5d-af82a6a3cdc7',
       'https://www.eenews.net/articles/oregon-lawmakers-ready-to-junk-contentious-wildfire-map/',
       'https://www.theguardian.com/us-news/article/2024/jul/27/oregon-wildfire-season',
       'https://www.opb.org/article/2025/05/24/oregon-fire-restrictions-washington-wildfire-bureau-land-management/',
       'https://apnews.com/article/oregon-wildfire-hazard-map-45c0335d93632580e07512a276dea7da',
       'https://www.kgw.com/article/news/local/wildfire/oregon-firefighters-return-north-carolina/283-7e13e4fc-10ad-498a-b4cb-85a5b0845603'
    ],
    "Maui_Kahikinui_Fire_Jun2025": [
      "https://www.theguardian.com/us-news/2025/jun/17/hawaii-maui-wildfire",
      'https://www.foxweather.com/extreme-weather/maui-brush-fire-kahikinui-hawaii-evacuations',
      'https://www.kitv.com/news/local/maui-county-hands-over-kahikinui-fire-control-to-state-of-hawaii/article_e453a5ab-90be-4d81-93d1-aeee87e5f989.html',
      'https://www.butlereagle.com/20250616/fast-moving-brush-fire-on-hawaiis-maui-island-evacuates-about-50-people-no-structures-have-burned/',
      'https://www.kcra.com/article/fast-moving-brush-fire-in-hawaiis-maui-county-evacuates-at-least-105-homes-no-structures-burned/65078861'
    ]
}


# Web scraper functions

In [ ]:
def extract_date(soup):
    selectors = [
      ('meta',{'itemprop':'datePublished'}),('meta',{'name':'pubdate'}),
      ('meta',{'name':'date'}),('meta',{'property':'article:published_time'}),
      ('span',{'class':'update-time'}),('p',{'class':'update-time'})
    ]
    for tag,attrs in selectors:
        el=soup.find(tag,attrs=attrs)
        if not el: continue
        s = el.get('content') if tag=='meta' and el.has_attr('content') else el.get_text(strip=True)
        try: return parser.parse(s,fuzzy=True).strftime('%Y-%m-%d')
        except: return s
    return "n/a"

def scrape_article(url):
    r = requests.get(url,headers={"User-Agent":"Mozilla/5.0"},timeout=10)
    r.raise_for_status()
    soup=BeautifulSoup(r.text,'html.parser')
    h1=soup.find('h1'); headline=h1.get_text(strip=True) if h1 else 'n/a'
    author='n/a'
    for tag,cls in [('span','byline__name'),('p','metadata__byline__author')]:
        el=soup.find(tag,class_=cls)
        if el: author=el.get_text(strip=True); break
    date = extract_date(soup)
    paras = soup.find_all('div',class_='paragraph') or soup.find_all('div',class_='zn-body__paragraph') or soup.find_all('p')
    content = "\n".join(p.get_text(" ",strip=True) for p in paras)
    return {"url":url,"headline":headline,"author":author,"date":date,"content":content}


# LLM call function

In [ ]:
NVIDIA_INVOKE_URL = "https://integrate.api.nvidia.com/v1/chat/completions"
def call_nvidia_llm(prompt):
    resp = requests.post(
      NVIDIA_INVOKE_URL,
      headers={"Authorization":f"Bearer {os.getenv('NVIDIA_API_KEY')}","Accept":"application/json"},
      json={
        "model":"nvidia/nemotron-mini-4b-instruct",
        "messages":[{"role":"user","content":prompt}],
        "max_tokens":1024,"temperature":0.2,"top_p":0.7
      }
    )
    resp.raise_for_status()
    return resp.json()["choices"][0]["message"]["content"]


# NeMo Embedder and Reranker

In [ ]:
# embedder & reranker
embedder = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5", truncate="END")
NV_rerank = NVIDIARerank(
  model="nvidia/nv-rerankqa-mistral-4b-v3",
  api_key=os.getenv("NVIDIA_API_KEY"),
  top_n=20
)


# Retriver

In [ ]:
# Retrieval helper
KEYWORDS = ["evacuate","rescue","firefighter","fire","water","emergency", "firebreak", "emergency", "shelter"]

def retrieve_top_k(query, sentences, embeddings, k=50, boost=0.5, pre_k=100):
    q_emb = np.array(embedder.embed_query(query))
    sims = embeddings @ q_emb / (np.linalg.norm(embeddings,1) * np.linalg.norm(q_emb))
    for i,s in enumerate(sentences):
        if any(kw in s.lower() for kw in KEYWORDS): sims[i]+=boost
    idxs=np.argsort(-sims)[:pre_k]
    cands=[sentences[i] for i in idxs]
    docs=[Document(page_content=t) for t in cands]
    try:
        out=NV_rerank.compress_documents(query=query,documents=docs)
        reranked = [d.page_content for d in (out["documents"] if isinstance(out,dict) else out)]
    except:
        reranked=cands
    return reranked[:k]


# SQL DB Schema

In [ ]:
conn = sqlite3.connect("incidents.db")
c = conn.cursor()
c.execute("""
CREATE TABLE IF NOT EXISTS incidents (
  id                          INTEGER PRIMARY KEY AUTOINCREMENT,
  incident_name               TEXT,
  incident_type               TEXT,
  severity_level              INTEGER,
  affected_location           TEXT,
  identified_cause            TEXT,
  estimated_economic_loss_usd INTEGER,
  reported_fatalities         INTEGER,
  reported_injuries           INTEGER,
  incident_summary            TEXT,
  response_measures           TEXT,
  anticipated_developments    TEXT,
  affected_population_estimate INTEGER,
  evacuations_ordered         TEXT,
  infrastructure_impacted     TEXT,
  emergency_status            TEXT,
  responding_agencies         TEXT
)
""")
conn.commit()


# LLM summarization and Loading Data into SQL

In [ ]:
# Process & store each incident
for name, urls in incidents.items():
    # scrape
    arts = [scrape_article(u) for u in urls]
    # build sentences+embeddings
    sents=[]
    for art in arts:
        sents+= [t.strip() for t in re.split(r'(?<=[.?!])\s+',art["content"])
                  if len(t.strip())>20 or re.search(r"\d",t)]
    embs = np.array(embedder.embed_documents(sents))
    context="\n".join(retrieve_top_k(
        "Summarize the key facts and response measures", sents, embs, k=50))
    prompt=f"""
You are an information‐extraction assistant.
Extract only **explicitly stated** facts into JSON—do **not** infer, assume, or hallucinate.
**Return only the JSON object, with no markdown fences, no explanations, and no extra text.**

Here is the schema your output must follow exactly:
{{
  "incident_name" : ""                      // Must be the name of incident in short, sort of like a title, TEXT.
  "incident_type": "",                      // Must be categorised into these categories only:  wildfire, urban fire, crowd-management incident, Undefined)
  "severity_level": 0,                      // 1–10 reflecting severity based on described impact; use 0 if unspecified
  "affected_location": "",                  // Region, city, state, etc.
  "identified_cause": "",                   // Cause if mentioned (Look for text which mentions reason for any calamitites or incidents) or mention undefined
  "estimated_economic_loss_usd": 0,         // Numeric, no formatting (e.g., 5000000)
  "reported_fatalities": 0,                 // Number of deaths of people if any or 0
  "reported_injuries": 0,                   // Number of injured of people if any or 0
  "incident_summary": "",                   // 1–2 sentence summary of the event. Must highlight main points related mostly to the incident impact and emergency responses
  "response_measures": "",                  // Steps taken in detail (e.g., evacuations, Incident-management, steps, declarations) by first responders or emergency agencies. Make sure to provide detailed information about the steps taken by the responders to handel the incident.
                                            // Provide a bullet numbered list of steps taken by the authorities to manage the incident and also manage the calamity as we need to find how they contained it. Avoid descriptive and unrealted words in response.
  "anticipated_developments": "",           // Future expectations or projections like if the threat is predicted to scale or it is mitigated
  "affected_population_estimate": 0,        // Check if they have mentioned any number of people affected/evacuated or rescured only peoples Not the homes/property etc damaged etc., Only number of people. If not found mention 0
  "evacuations_ordered": "",                // "Yes", "No", or "n/a"
  "infrastructure_impacted": "",            // E.g., roads, power lines, water
  "emergency_status": "",                   // "Declared", "Not Declared", or "n/a"
  "responding_agencies": "",                // Government or official organizations mentioned
}}

Context:
{context}
""".strip()
    raw = call_nvidia_llm(prompt)
    rec = json.loads(re.search(r"\{[\s\S]*\}",raw).group(0))
    rec["incident_name"]=name
    cols=list(rec.keys())
    ph=",".join("?" for _ in cols)
    sql=f"INSERT INTO incidents ({','.join(cols)}) VALUES ({ph})"
    c.execute(sql,[rec[c] for c in cols])
    conn.commit()
    print(f"Stored {name} as id={c.lastrowid}")


Stored Oregon_Rowena_Fire_Jun2025 as id=1
Stored Maui_Kahikinui_Fire_Jun2025 as id=2


In [ ]:
conn.close()

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.8 MB/s eta 0:00:00


# Vectorize SQL to DB

In [ ]:
import sqlite3
import numpy as np
import faiss
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

conn = sqlite3.connect("incidents.db")
cur  = conn.cursor()
cur.execute("SELECT id, incident_summary, response_measures FROM incidents")
rows = cur.fetchall()
conn.close()

ids       = [r[0] for r in rows]
documents = [f"{r[1]} {r[2]}" for r in rows]

print(f"Loaded {len(ids)} incidents from SQLite.")

embedder = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5", truncate="END")
vectors  = np.array(embedder.embed_documents(documents), dtype="float32")

d     = vectors.shape[1]
index = faiss.IndexFlatIP(d)
faiss.normalize_L2(vectors)
index.add(vectors)

print(f"✅ FAISS index built with {index.ntotal} vectors.")


Loaded 2 incidents from SQLite.
✅ FAISS index built with 2 vectors.


In [ ]:
def recommend_wildfire_control(query: str, top_k: int = 5):
    q_vec = np.array(embedder.embed_query(query), dtype="float32").reshape(1, -1)
    faiss.normalize_L2(q_vec)
    D, I = index.search(q_vec, top_k)
    conn = sqlite3.connect("incidents.db")
    cur  = conn.cursor()
    contexts = []
    for idx in I[0]:
        rec_id = ids[idx]
        cur.execute(
            "SELECT incident_summary, response_measures FROM incidents WHERE id = ?",
            (rec_id,)
        )
        summary, measures = cur.fetchone()
        contexts.append(f"Summary: {summary}\nMeasures: {measures}")
    conn.close()

    # 4) Build the LLM prompt
    combined_ctx = "\n\n---\n\n".join(contexts)
    prompt = f"""
You are an emergency response planning assistant helping the first responders and emergency agencies at the critical scene.
Below are summaries and response measures from {top_k} past wildfire incidents:
{combined_ctx}

Given the new scenario:
\"\"\"{query}\"\"\"

Draft a JSON object with a key "recommended_steps" whose value is an array of concise, numbered best-practice steps to control this wildfire.
Return **only** the JSON object, no extra text.
"""
    # 5) Call the NVIDIA LLM
    raw = call_nvidia_llm(prompt)
    # 6) Extract and parse JSON
    m = re.search(r"\{{[\s\S]*\}}", raw)
    js = m.group(0) if m else raw
    return json.loads(js)

# Example usage:
query_text = "There is a wildfire in California with heavy vegetation—what steps can be taken to control it?"
recommendations = recommend_wildfire_control(query_text, top_k=5)
print(json.dumps(recommendations, indent=2))


{
  "recommended_steps": [
    "Implement a containment line to prevent the fire from spreading further.",
    "Use water-dropping aircraft and helicopters to douse hotspots and slow the fire's spread.",
    "Implement a backfiring strategy to create a firebreak and reduce the fuel load.",
    "Use hand crews to build a fireline and maintain the containment line.",
    "Monitor the fire's progress and adjust strategies as needed.",
    "Communicate with local communities and provide evacuation orders if necessary.",
    "Coordinate with other agencies and jurisdictions to share resources and information."
  ]
}


In [ ]:
# # Cell 9: Query the vector store for “control steps” advice

# def get_control_steps(query: str, top_k: int = 3):
#     # 1) Embed & normalize the query
#     q_vec = np.array(embedder.embed_query(query), dtype="float32").reshape(1, -1)
#     faiss.normalize_L2(q_vec)
#     # 2) Search FAISS
#     D, I = index.search(q_vec, top_k)
#     # 3) Retrieve and display the top matching incidents’ steps
#     conn = sqlite3.connect("incidents.db")
#     cur  = conn.cursor()
#     for score, idx in zip(D[0], I[0]):
#         rec_id = ids[idx]
#         cur.execute("SELECT incident_summary, response_measures FROM incidents WHERE id = ?", (rec_id,))
#         summary, measures = cur.fetchone()
#         print(f"---\nIncident ID: {rec_id}  (score {score:.3f})")
#         print("Summary:\n", summary, "\n")
#         print("Response Measures:\n", measures, "\n")
#     conn.close()

# # Example usage:
# get_control_steps(
#     "There is a wildfire in California with heavy vegetation—what steps can be taken to control it?"
# )


---
Incident ID: 1  (score 0.376)
Summary:
 A wildfire in Oregon prompted officials to issue evacuation orders for hundreds of homes and to close nearly 20 miles of an interstate in the Columbia River Gorge. 

Response Measures:
 A water helicopter and a plane dropping fire retardant helped fight the fire, which broke out on Wednesday. Firefighters from 10 states have come to Oregon to assist with suppression efforts. 

---
Incident ID: 2  (score 0.356)
Summary:
 Firefighters successfully contained the blaze, but 105 homes were evacuated due to strong winds and fast-moving fire. 

Response Measures:
 Firefighters, emergency officials, and the Hawaii National Guard deployed resources to battle the blaze. Evacuations were conducted, and part of a highway was closed. The American Red Cross provided shelters for displaced residents. 

---
Incident ID: 2  (score -340282346638528859811704183484516925440.000)
Summary:
 Firefighters successfully contained the blaze, but 105 homes were evacuate